# Assignment 8 - Convolutional Neural Networks

Congratulations! You have made it to the last assignment of the TechX Machine Learning bootcamp. In the following assignment, you will explore with convolutional neural networks (CNNs) by training a classifier to recognize the difference between images of cats and dogs. The training time of this assignment will take longer than previous ones, so it reccommended to run this on a GPU.

Start by download the dataset to your local environment: URL HERE

### Import Libraries

As usual, let's start by importing the necessary libaries.

In [1]:
from __future__ import print_function
from keras.models import Model, Sequential, load_model
from keras.layers import Dense, Input, Conv2D, MaxPooling2D, Flatten, Dropout
from keras.datasets import mnist
from keras.optimizers import Adam
from keras.preprocessing import image
from keras.applications import VGG16
from keras.preprocessing.image import ImageDataGenerator
# from vis.utils import utils
from keras import activations

import matplotlib as mpl
mpl.use('Agg')
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
import os
import pickle

/home/john/anaconda3/envs/pokemon/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### Helper Methods
Below are some helper methods that have been created for usage later in the assignment.

In [2]:
def smooth_curve(points, factor=0.8):
    smoothed = []
    for point in points:
        if smoothed:
            previous = smoothed[-1]
            smoothed.append(previous * factor + point * (1 - factor))
        else:
            smoothed.append(point)
    return smoothed

def plot_compare(history, steps=-1):
    if steps < 0:
        steps = len(history.history['acc'])
    acc = smooth_curve(history.history['acc'][:steps])
    val_acc = smooth_curve(history.history['val_acc'][:steps])
    loss = smooth_curve(history.history['loss'][:steps])
    val_loss = smooth_curve(history.history['val_loss'][:steps])
    
    plt.figure(figsize=(6, 4))
    plt.plot(loss, c='#0c7cba', label='Train Loss')
    plt.plot(val_loss, c='#0f9d58', label='Val Loss')
    plt.xticks(range(0, len(loss), 5))
    plt.xlim(0, len(loss))
    plt.title('Train Loss: %.3f, Val Loss: %.3f' % (loss[-1], val_loss[-1]), fontsize=12)
    plt.legend()
    
    plt.figure(figsize=(6, 4))
    plt.plot(acc, c='#0c7cba', label='Train Acc')
    plt.plot(val_acc, c='#0f9d58', label='Val Acc')
    plt.xticks(range(0, len(acc), 5))
    plt.xlim(0, len(acc))
    plt.title('Train Accuracy: %.3f, Val Accuracy: %.3f' % (acc[-1], val_acc[-1]), fontsize=12)
    plt.legend()
    
def deprocess_image(x):
    # normalize tensor: center on 0., ensure std is 0.1
    x -= x.mean()
    x /= (x.std() + 1e-5)
    x *= 0.1

    # clip to [0, 1]
    x += 0.5
    x = np.clip(x, 0, 1)

    # convert to RGB array
    x *= 255
    x = np.clip(x, 0, 255).astype('uint8')
    return x
 
def save_history(history, fn):
    with open(fn, 'wb') as fw:
        pickle.dump(history.history, fw, protocol=2)

def load_history(fn):
    class Temp():
        pass
    history = Temp()
    with open(fn, 'rb') as fr:
        history.history = pickle.load(fr)
    return history

def jitter(img, amount=32):
    ox, oy = np.random.randint(-amount, amount+1, 2)
    return np.roll(np.roll(img, ox, -1), oy, -2), ox, oy

def reverse_jitter(img, ox, oy):
    return np.roll(np.roll(img, -ox, -1), -oy, -2)

def plot_image(img):
    plt.figure(figsize=(6, 6))
    plt.imshow(img)
    plt.axis('off')

### Question 1: Load Data and Prepare Data

When it comes to CNNs, we are usually working with data that takes up a large amount of space which means we will need to load the data in batches. You'll also notice that the dataset only has 1500 images for each class, which is not nearly enough data to train an accurate classifier using a deep neural network approach. As a result, we will need to augment our data as well - perform linear transformations on the existing data in order to create additional new data.

Luckily, there is [built in functionality in keras](https://keras.io/preprocessing/image/) to help us load our data in batches while augmenting it in real-time. Let's start by using this library to load and prepare our data.

#### Start by setting the root path of your dataset:

In [3]:
## ========== YOUR CODE HERE ========= ##
base_dir = 'dataset'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'valid')
test_dir = os.path.join(base_dir, 'test')

#### Create instances of the ImageDataGenerator object for the train and test data with the following parameters:
1) Train data:
- Rescale: 1./255
- Rotation Range: 40
- Width Shift Range: 0.2
- Height Shift Range: 0.2
- Shear Range: 0.2
- Zoom Range: 0.2
- Horizontal Flip: True

2) Test data:
- Rescale: 1./255

#### Use the online documentation for support.

In [4]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,)

test_datagen = ImageDataGenerator(rescale=1./255)

#### Set the directory, target size (150,150), batch size (32), and class mode (binary) on the data generator instances using the .flow_from_directory method. 

In [5]:
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(150, 150),
        batch_size=32,
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=32,
        class_mode='binary')

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


### Question 2: Build and Train Custom Model
Now that our data is ready, it is time to start training our model. This section will first go over an example of how to build a custom CNN architecture and train our data on this model. Then you will be given the outline for another custom architecture which you will have to build and train by yourself. 

#### For the first custom model, we will create and compile a sequential model with the following architecture
- Convolution layer with 32 filters, 3x3 kernel, relu activation, and padding
- Max pooling layer with a size of 2x2
- Convolution layer with 64 filters, 3x3 kernel, relu activation, and padding
- Max pooling layer with a size of 2x2
- Convolution layer with 128 filters, 3x3 kernel, relu activation, and padding
- Max pooling layer with a size of 2x2
- Convolution layer with 128 filters, 3x3 kernel, relu activation, and padding
- Max pooling layer with a size of 2x2
- Flattening layer
- Dropout layer with 0.5 dropout rate
- Densely connected layer with dimension of 512 and relu activation function
- Densely connected layer with dimension of 256 and relu activation function
- Densely connected output layer with dimension of 1 and sigmoid activation function

In [7]:
# create the model
model_custom = Sequential()
model_custom.add(Conv2D(32, (3, 3), activation='relu', padding='same', name='conv_1', 
                 input_shape=(150, 150, 3)))
model_custom.add(MaxPooling2D((2, 2), name='maxpool_1'))
model_custom.add(Conv2D(64, (3, 3), activation='relu', padding='same', name='conv_2'))
model_custom.add(MaxPooling2D((2, 2), name='maxpool_2'))
model_custom.add(Conv2D(128, (3, 3), activation='relu', padding='same', name='conv_3'))
model_custom.add(MaxPooling2D((2, 2), name='maxpool_3'))
model_custom.add(Conv2D(128, (3, 3), activation='relu', padding='same', name='conv_4'))
model_custom.add(MaxPooling2D((2, 2), name='maxpool_4'))
model_custom.add(Flatten())
model_custom.add(Dropout(0.5))
model_custom.add(Dense(512, activation='relu', name='dense_1'))
model_custom.add(Dense(256, activation='relu', name='dense_2'))
model_custom.add(Dense(1, activation='sigmoid', name='output'))

# compile the model
model_custom.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

#### We can print a summary of the architecture as shown below

In [8]:
model_custom.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_1 (Conv2D)              (None, 150, 150, 32)      896       
_________________________________________________________________
maxpool_1 (MaxPooling2D)     (None, 75, 75, 32)        0         
_________________________________________________________________
conv_2 (Conv2D)              (None, 75, 75, 64)        18496     
_________________________________________________________________
maxpool_2 (MaxPooling2D)     (None, 37, 37, 64)        0         
_________________________________________________________________
conv_3 (Conv2D)              (None, 37, 37, 128)       73856     
_________________________________________________________________
maxpool_3 (MaxPooling2D)     (None, 18, 18, 128)       0         
_________________________________________________________________
conv_4 (Conv2D)              (None, 18, 18, 128)       147584    
__________

#### We can now train and save the model.

In [9]:
#fit model
history_custom = model_custom.fit_generator(train_generator, steps_per_epoch=100, epochs=20, 
                                      validation_data=validation_generator, validation_steps=50, verbose=1)

# # save model
model_custom.save('model-custom.h5')
save_history(history_custom, 'history_custom.bin')

# plot results
history_aug = load_history('history_custom.bin')
plot_compare(history_custom)

Epoch 1/20
  3/100 [..............................] - ETA: 4:23 - loss: 0.6836 - acc: 0.5208

KeyboardInterrupt: 

In [17]:
# plot results
history_aug = load_history('history_custom.bin')
plot_compare(history_custom)

### Question 3: Build and Train VGG Model

### Question 4: Visualize Feature Maps

### Question 5: Visualize Dense Layer

### Question 6: Visualize Convolutional Filters